<a href="https://colab.research.google.com/github/triakshunn/Sentiment-Analysis-for-twitter-data/blob/main/IITK_Sentiment_Analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentiment Analysis model using LSTM,CNN's and w2v

Importing necessary libraries

In [ ]:
import tensorflow as tf
import re
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import gensim
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, LSTM
from keras import utils
from keras import layers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

Loading the data

In [ ]:
train_data=pd.read_csv('/content/train_data_1.csv')
train_data

,tweet_id,sentiment,tweet_text
0,2.641838e+17,positive,Gas by my house hit $3.39!!!! I\u2019m going t...
1,2.634051e+17,negative,Theo Walcott is still shit\u002c watch Rafa an...
2,2.621632e+17,negative,its not that I\u2019m a GSP fan\u002c i just h...
3,2.642493e+17,negative,Iranian general says Israel\u2019s Iron Dome c...
4,2.626820e+17,neutral,Tehran\u002c Mon Amour: Obama Tried to Establi...
...,...,...,...
21460,5.229490e+17,neutral,"the day after newark ill be able to say """"i me..."
21461,5.223726e+17,neutral,FEC hold farewell session for seven ministers ...
21462,5.225152e+17,neutral,Luca Di Montezemolo (who's last day was Monday...
21463,5.230891e+17,positive,Coffee is pretty much the answer to all questi...


In [ ]:
test_data=pd.read_csv('/content/Test_data_IITK.csv')
test_data

,tweet_id,tweet_text
0,264238274963451904,"@jjuueellzz down in the Atlantic city, ventnor..."
1,218775148495515649,Musical awareness: Great Big Beautiful Tomorro...
2,258965201766998017,On Radio786 100.4fm 7:10 Fri Oct 19 Labour ana...
3,262926411352903682,"Kapan sih lo ngebuktiin,jan ngomong doang Susa..."
4,171874368908050432,"Excuse the connectivity of this live stream, f..."
...,...,...
5393,210378118865756160,It's a Wednesday girls night out as '90's band...
5394,245177521304399872,"night college course sorted, just have to enro..."
5395,259280987089932288,For the 1st time in 30 years. For your splendi...
5396,201113950211940352,NURSES DAY - 12 MAY 2012. Nursing: The heart b...


Data pre-processing starts

In [ ]:
def convert_text_to_number(dataset,column):
     dataset[str(column)]=dataset[str(column)].apply(lambda x:re.sub('positive','0',x))
     dataset[str(column)]=dataset[str(column)].apply(lambda x:re.sub('negative','1',x))
     dataset[str(column)]=dataset[str(column)].apply(lambda x:re.sub('neutral','2',x))

In [ ]:
convert_text_to_number(train_data,'sentiment')
train_data

,tweet_id,sentiment,tweet_text
0,2.641838e+17,0,Gas by my house hit $3.39!!!! I\u2019m going t...
1,2.634051e+17,1,Theo Walcott is still shit\u002c watch Rafa an...
2,2.621632e+17,1,its not that I\u2019m a GSP fan\u002c i just h...
3,2.642493e+17,1,Iranian general says Israel\u2019s Iron Dome c...
4,2.626820e+17,2,Tehran\u002c Mon Amour: Obama Tried to Establi...
...,...,...,...
21460,5.229490e+17,2,"the day after newark ill be able to say """"i me..."
21461,5.223726e+17,2,FEC hold farewell session for seven ministers ...
21462,5.225152e+17,2,Luca Di Montezemolo (who's last day was Monday...
21463,5.230891e+17,0,Coffee is pretty much the answer to all questi...


In [ ]:
train_data=train_data.drop('tweet_id',axis=1)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21465 entries, 0 to 21464
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sentiment   21465 non-null  object
 1   tweet_text  21465 non-null  object
dtypes: object(2)
memory usage: 335.5+ KB


Cleaning the data

In [ ]:

import nltk
nltk.download('stopwords')
stop_words=stopwords.words("english")
stop_words=stop_words[:116] # #Left the rest of the words because they include 'don't','not' etc which may help deciding polarisibilty.
stop_words
stemmer=SnowballStemmer("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
urlPattern= r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
userPattern= '@[^\s]+'
alphaPattern= "[^a-zA-Z0-9]"
sequencePattern= r"(.)\1\1+"
seqReplacePattern= r"\1\1"
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}


In [ ]:
def data_cleaning(text,stem=False):
    for emoji in emojis.keys():
        text=text.replace(emoji,"EMOJI"+emojis[emoji])
    text=re.sub(urlPattern,' URL',str(text).lower()).strip()
    text=re.sub(userPattern,' USER',str(text).lower()).strip()
    text=re.sub(alphaPattern,' ',str(text).lower()).strip()
    text=re.sub(sequencePattern,seqReplacePattern,str(text).lower()).strip()
    new_text=[]
    for texts in text.split():
        if texts not in stop_words:
            if stem:
                new_text.append(stemmer.stem(texts))
            else:
                new_text.append(texts)
    return " ".join(new_text)            
            

In [ ]:
train_data["tweet_text"]=train_data["tweet_text"].apply(lambda x: data_cleaning(x))

In [ ]:
test_data["tweet_text"]=test_data["tweet_text"].apply(lambda x: data_cleaning(x))

w2v starts here

Defining the constants

In [ ]:
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

In [ ]:
documents = [_text.split() for _text in train_data.tweet_text] 


In [ ]:
w2v_model = gensim.models.word2vec.Word2Vec(size=W2V_SIZE, 
                                            window=W2V_WINDOW, 
                                            min_count=W2V_MIN_COUNT,
                                            workers=8)

In [ ]:
w2v_model.build_vocab(documents)


In [ ]:
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size", vocab_size)


Vocab size 3969


In [ ]:
%%time
w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)


CPU times: user 32.7 s, sys: 70.1 ms, total: 32.8 s
Wall time: 17.3 s


(6796010, 9385280)

In [ ]:
w2v_model.most_similar("love")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('xx', 0.44422441720962524),
 ('x', 0.4244207441806793),
 ('appreciate', 0.42357149720191956),
 ('poetic', 0.4039784073829651),
 ('thought', 0.39778438210487366),
 ('stan', 0.39643192291259766),
 ('thank', 0.3956957161426544),
 ('hate', 0.3933345079421997),
 ('zayn', 0.3848564922809601),
 ('mind', 0.38388752937316895)]

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(train_data["tweet_text"])
vocabulary_size = len(tokenizer.word_index) + 1
vocabulary_size

28290

In [ ]:
SEQUENCE_LENGTH = 300

In [ ]:
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data.tweet_text), maxlen=SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data.tweet_text), maxlen=SEQUENCE_LENGTH)
x_test.shape

(5398, 300)

In [ ]:
from keras.utils.np_utils import to_categorical
y_train=train_data.sentiment
y_train_oh = to_categorical(y_train)
y_train_oh

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [ ]:
embedding_matrix = np.zeros((vocabulary_size, W2V_SIZE))
for word, i in tokenizer.word_index.items():
  if word in w2v_model.wv:
    embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)


(28290, 300)


In [ ]:
embedding_layer = Embedding(vocabulary_size, W2V_SIZE, weights=[embedding_matrix], input_length=SEQUENCE_LENGTH, trainable=False)


w2v training ends here;Also data has been processed to feed to our model.

Training starts here

In [ ]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(LSTM(200,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(units = 150, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 300, 300)          8487000   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 300, 32)           28832     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 150, 32)           0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 150, 32)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 200)               186400    
_________________________________________________________________
dense_7 (Dense)              (None, 150)               30150     
_________________________________________________________________
dropout_8 (Dropout)          (None, 150)              

In [ ]:
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
             EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5)]

In [ ]:
EPOCHS = 20
BATCH_SIZE = 1024

In [ ]:
%%time
history = model.fit(x_train, y_train_oh,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_split=0.05,
                    verbose=1,
                    callbacks=callbacks)

Train on 20391 samples, validate on 1074 samples
Epoch 1/20
20391/20391 [==============================] - 12s 586us/step - loss: 0.6082 - accuracy: 0.6593 - val_loss: 0.5506 - val_accuracy: 0.7272
Epoch 2/20
20391/20391 [==============================] - 11s 539us/step - loss: 0.5588 - accuracy: 0.7074 - val_loss: 0.5180 - val_accuracy: 0.7418
Epoch 3/20
20391/20391 [==============================] - 11s 542us/step - loss: 0.5334 - accuracy: 0.7279 - val_loss: 0.5042 - val_accuracy: 0.7461
Epoch 4/20
20391/20391 [==============================] - 11s 540us/step - loss: 0.5239 - accuracy: 0.7345 - val_loss: 0.5015 - val_accuracy: 0.7502
Epoch 5/20
20391/20391 [==============================] - 11s 549us/step - loss: 0.5169 - accuracy: 0.7386 - val_loss: 0.4986 - val_accuracy: 0.7523
Epoch 6/20
20391/20391 [==============================] - 11s 541us/step - loss: 0.5104 - accuracy: 0.7459 - val_loss: 0.4981 - val_accuracy: 0.7542
Epoch 7/20
20391/20391 [==============================] -

Training ended

Code for predicting the output

In [ ]:
def predict(model):
    
    
    history=model.predict(x_test
                         ,batch_size=BATCH_SIZE
                         ,callbacks=callbacks
                         ,verbose=1)
    return history


In [ ]:
output=predict(model)
print(len(predict(model)))

5398/5398 [==============================] - 1s 157us/step
5398


In [ ]:
array_init=np.argmax(output, axis=1)  # using argmax to get output from one hot encode to '0','1','2'.
array_init=list(array_init)

In [ ]:
for i in range(len(predict(model))): #converting numbers to positive,negative,neutral.
    if array_init[i]==0:
        array_init[i]='positive'
    elif array_init[i]==1:
        array_init[i]='negative'
    elif array_init[i]==2:
        array_init[i]='neutral'
array_init

5398/5398 [==============================] - 1s 156us/step


['neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'positive',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 '

Converting the output to pandas dataframe for evaluation in Kaggle

In [ ]:
output = pd.DataFrame({'tweet_id': test_data.tweet_id, 'sentiment': array_init})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
